In [27]:
from dotenv import dotenv_values
from openai import OpenAI
import PyPDF2
import re
import json


In [28]:
env_vars=dotenv_values(".env")
OPENAI_API_KEY=env_vars.get("OPENAI_API_KEY")


In [29]:
if OPENAI_API_KEY:
  try:
    openai_client=OpenAI(api_key=OPENAI_API_KEY)
    openai_client.models.list()
  except Exception as e:
    print (f"Incorrect Key{e}")
    

In [30]:
def extract_text_from_pdf(pdf_filename):
    text = ""
    try:
        with open(pdf_filename, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return text

In [31]:
def fix_number_format(text):

    # Convert numbers formatted as "xxx,xxx.xxx" to "xxxxxx.xxx"
    text = re.sub(r'(\d{1,3}),(\d{3}\.\d+)', r'\1\2', text)  

    return text

In [32]:
pdf_filename = "Thermo Fisher Purchase Order NEU - 0007450772 multiple lines.pdf"
pdf_text = extract_text_from_pdf(pdf_filename)
pdf_text = fix_number_format(pdf_text)


In [33]:
system_message = (
    "You are an AI extracting relevant content from a purchase order. "
    "Find the following details and return ONLY a valid JSON object with these fields:"
    "\n- Customer Name (Look for terms and condition and header section)"
    "\n- Purchase Order Number"
    "\n- Required Delivery Date (convert to ISO format YYYY-MM-DD)" 
    "\n- Material Number (Extract from the line item section, usually in the same row as 'Order Qty' and 'UOM')"
    "\n- Order Quantity in kg (only the converted kg value, do not include pounds or extra text, round to the nearest integer)"
    "\n- Delivery Address (extract ONLY the 'SHIP TO' address, includes distribution name if it is there, ignore all other addresses including 'Vendor', 'Invoice', 'Billing', and any address containing 'PO Box')"
    "\n\nIMPORTANT: "
    "- Return ONLY a valid JSON object. Do NOT include explanations, introductions, or Markdown formatting."
    "- Ensure 'Order Quantity in kg' is a clean number without thousand separators or extra text."
    "- Ensure 'Required Delivery Date' follows ISO 8601 format (YYYY-MM-DD)."
    "- Ensure 'Delivery Address' is the correct 'SHIP TO' address."
    "- Ignore addresses related to 'Vendor', 'Invoice', 'Billing', 'Remit To', 'PO Box', or 'Mailing Address'."
    "- Ignore Material Number related to 'Vendor', 'Invoice', 'Billing', 'Remit To', 'PO Box', or 'Mailing "
    "- Ignore **Price per unit** label."
)




user_prompt=f"Extract relevant details from the following purchase order:\n{pdf_text}"
prompts=[
  {"role":"system","content":system_message},
  {"role":"user","content":user_prompt},
]

prompts.append({
    "role": "system",
    "content": (
        "Analyze the purchase order details provided. If the item section contains more than one item number, this indicates there are multiple purchase order lines. "
        "In that case, extract and output each line separately as a JSON array, where each element represents a single purchase order line with all its details. "
        "If there's only one line, output it as a single JSON object. Ensure that no line is omitted."
    )
})




In [34]:
MODEL='gpt-4o'
response=openai_client.chat.completions.create(
  model=MODEL,
  messages=prompts,
  temperature=0,
   top_p=0.1,
)
print(response.choices[0].message.content)
extract_contents=response.choices[0].message.content


```json
[
    {
        "Customer Name": "Nalge Nunc International",
        "Purchase Order Number": "7450772",
        "Required Delivery Date": "2025-04-02",
        "Material Number": "8-0056-35",
        "Order Quantity in kg": 21773,
        "Delivery Address": "Penfield Plant, Nalge Nunc International, 75 Panorama Creek Drive, ROCHESTER NY 14625-2303"
    },
    {
        "Customer Name": "Nalge Nunc International",
        "Purchase Order Number": "7450772",
        "Required Delivery Date": "2025-04-16",
        "Material Number": "8-0056-35",
        "Order Quantity in kg": 21773,
        "Delivery Address": "Penfield Plant, Nalge Nunc International, 75 Panorama Creek Drive, ROCHESTER NY 14625-2303"
    },
    {
        "Customer Name": "Nalge Nunc International",
        "Purchase Order Number": "7450772",
        "Required Delivery Date": "2025-04-30",
        "Material Number": "8-0056-35",
        "Order Quantity in kg": 21773,
        "Delivery Address": "Penfield Plant,

In [35]:
# Convert string to JSON format
try:
    extract_contents_json = json.loads(extract_contents)  # Convert to dictionary
    print("Valid JSON Output:\n", json.dumps(extract_contents_json, indent=4))  # Pretty print JSON
except json.JSONDecodeError as e:
    print("Error: OpenAI did not return valid JSON.\n", e)
    print("Raw API Response:\n", extract_contents)

Error: OpenAI did not return valid JSON.
 Expecting value: line 1 column 1 (char 0)
Raw API Response:
 ```json
[
    {
        "Customer Name": "Nalge Nunc International",
        "Purchase Order Number": "7450772",
        "Required Delivery Date": "2025-04-02",
        "Material Number": "8-0056-35",
        "Order Quantity in kg": 21773,
        "Delivery Address": "Penfield Plant, Nalge Nunc International, 75 Panorama Creek Drive, ROCHESTER NY 14625-2303"
    },
    {
        "Customer Name": "Nalge Nunc International",
        "Purchase Order Number": "7450772",
        "Required Delivery Date": "2025-04-16",
        "Material Number": "8-0056-35",
        "Order Quantity in kg": 21773,
        "Delivery Address": "Penfield Plant, Nalge Nunc International, 75 Panorama Creek Drive, ROCHESTER NY 14625-2303"
    },
    {
        "Customer Name": "Nalge Nunc International",
        "Purchase Order Number": "7450772",
        "Required Delivery Date": "2025-04-30",
        "Material Nu